In [ ]:
from grewpy import Request
import re

def pattern_to_request(pattern, scope):
    """
    Build a Grew request from a Grex pattern
    """
    def parents_in_scope(scope: str) -> dict:
        """
        Get scope dependencies. Parent relations are needed to build a grew request.
        """
        parents = dict()
        for clause in Request(scope).json_data():
            for constraint in clause['pattern']: # type: ignore
                if "->" in constraint:
                    parent, child = re.split("-.*-?>", constraint)
                    parents[child] = parent
        return parents

    scope_parents = parents_in_scope(scope)
    request = Request()

    for clause in re.split(",node", pattern):
        _, node_name, target, feature = clause.split(":", maxsplit=3)
        keyword = "with" if target == "child" else "pattern"
        feat, value = feature.split("=")
        parent = scope_parents.get(node_name, f"{node_name}parent")
        # position
        if feat == "position":
            if value == "after":
                request.append(keyword, f"{parent}->{node_name}; {node_name} << {parent}")
            else:
                request.append(keyword, f"{parent}->{node_name}; {parent} << {node_name}")
        # deprel
        elif "rel_shallow" in feat:
            if target == "own":
                request.append(keyword, f'{parent}-[{value}]->{node_name}')
            else: #child
                request.append(keyword, f'{node_name}-[{value}]->{node_name}child')
        elif target == "child":
            request.append(keyword, f'{node_name}->{node_name}child; {node_name}child[{feat}="{value}"]')
        elif target == "parent":
            request.append(keyword, f'{parent}->{node_name}; {parent}[{feat}="{value}"]')
        else: #own
            request.append(keyword, f'{node_name}[{feat}="{value}"]')
    lst_request = [str(req) for req in request]
    return "\n".join(lst_request)

In [13]:
import pandas as pd
import json

with open("rules_subject_inversion.json") as f:
    data = json.load(f)

scope = "pattern { X-[1=subj]->Y}"

df = pd.DataFrame(data['rules'])
df['pattern'] = df['pattern'].map(lambda x: pattern_to_request(x, scope))
df.round(3)

,pattern,n_pattern_occurence,n_pattern_positive_occurence,decision,alpha,value,coverage,precision,delta,g-statistic,p-value,cramers_phi
0,with {X-[parataxis]->Xchild},97,36,yes,0.018,0.407,55.385,37.113,32.302,112.007,0.000,1.075
1,with {X-[comp:obj]->Xchild},608,9,no,0.006,-0.087,36.524,98.520,-14.179,11.670,0.001,0.139
2,"with {X->Xchild;Xchild[xpos=""IN""]}",626,10,no,0.006,-0.077,37.561,98.403,-13.865,10.650,0.001,0.130
3,"with {X->Xchild;Xchild[xpos=""NN""]}",423,6,no,0.005,-0.003,25.427,98.582,-10.126,8.638,0.003,0.143
4,"pattern {Y[xpos=""NNP""]}",239,22,yes,0.004,0.119,33.846,9.205,12.889,13.746,0.000,0.240
5,"pattern {Y[Number=""Sing""]}",1093,54,yes,0.004,0.036,83.077,4.941,12.331,3.480,0.062,0.056
6,"with {Y->Ychild;Ychild[upos=""NOUN""]}",80,12,yes,0.004,0.201,18.462,15.000,8.950,16.059,0.000,0.448
7,"with {X->Xchild;Xchild[xpos=""RB""]}",370,5,no,0.004,-0.041,22.256,98.649,-9.106,8.071,0.004,0.148
8,"with {X->Xchild;Xchild[VerbForm=""Part""]}",665,20,no,0.004,-0.093,39.329,96.992,-5.352,1.264,0.261,0.044
9,"with {X->Xchild;Xchild[Gender=""Masc""]}",900,26,no,0.004,-0.065,53.293,97.111,-8.311,2.278,0.131,0.050
